In [1]:
import requests
import config
import pandas as pd
import numpy as np
from datetime import date
import pickle
import time
import sqlite3

In [89]:
URL = {
    'base': 'https://{proxy}.api.riotgames.com/lol/{url}',
    'summonerData': 'summoner/v{version}/summoners/by-name/{sumName}?api_key={apiKey}',
    'entryData': 'league/v{version}/entries/{queueType}/{tier}/{rank}?api_key={apiKey}',
    'leagueData': 'league/v{version}/leagues/{league}?api_key={apiKey}',
    'matchListData': 'match/v{version}/matchlists/by-account/{accountId}?api_key={apiKey}'
}

VERSION = {
    'summonerData': '4',
    'leagueData': '4',
    'entryData': '4',
    'matchListData': '4'
}

In [3]:
queueTypes = ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']
tiers = {'DIAMOND': 1, 'PLATINUM': 2, 'GOLD': 3, 'SILVER': 4, 'BRONZE': 5, 'IRON': 6}
ranks = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}
errorCodes = {400: 'Bad request', 401: 'Unauthorized', 403: 'Forbidden',
             404: 'Data not found', 405: 'Method not allowed', 415: 'Unsupported media type',
             429: 'Rate limit exceeded', 500: 'Internal server error', 502: 'Bad gateway',
             503: 'Service unavailable', 504: 'Gateway timeout'}

In [90]:
class riotApi(object):
    
    def __init__(self, apiKey, region = 'na1'):
        self.apiKey = apiKey
        self.region = region
        
    def _request(self, apiUrl, params = {}):
        args = {'apiKey': self.apiKey}
        for key, value in params.items():
            if key not in args:
                args[key] = value
        response = requests.get(
            URL['base'].format(
                proxy = self.region,
                url = apiUrl),
            params = args)
        request = response.json()
        
        try:
            code = request['status']['status_code']
        except KeyError:
            return request
        except TypeError:
            return request
        
        if code == 429: 
            print("Waiting 2 minutes for new requests...")
            time.sleep(60*2)
            self._request(apiUrl, params = {})
        else:
            raise RuntimeError(errorCodes[code])
    
    def summonerData(self, name):
        apiUrl = URL['summonerData'].format(
            version = VERSION['summonerData'],
            sumName = name,
            apiKey = self.apiKey)
        return self._request(apiUrl)
    
    def entryData(self, queueType, tier, rank):
        apiUrl = URL['entryData'].format(
            version = VERSION['entryData'],
            queueType = queueType,
            tier = tier,
            rank = rank,
            apiKey = self.apiKey)
        return self._request(apiUrl)
    
    def leagueData(self, league):
        apiUrl = URL['leagueData'].format(
            version = VERSION['leagueData'],
            league = league,
            apiKey = self.apiKey)
        return self._request(apiUrl)
    
    def matchListData(self, accountId):
        apiUrl = URL['matchListData'].format(
            version = VERSION['matchListData'],
            accountId = accountId,
            apiKey = self.apiKey)
        return self._request(apiUrl)
        

In [81]:
class sqlDatabase():
    
    def __init__(self, path):
        self.path = path
    
    def _open(self):
        self.db = sqlite3.connect(self.path)
        self.cursor = self.db.cursor()
        
    def _close(self):
        self.cursor.close()
        self.db.close()
        
    def appendData(self, df, table):
        self._open()
        
        try:
            # Load data from sql
            query = pd.read_sql_query(f"SELECT * FROM {table};", self.db)

            # Merge old with new data and remove duplicates
            df = query.append(df, ignore_index = True)
            print(f"{sum(df.duplicated())} duplicates deleted...")
            df.drop_duplicates(ignore_index = True, inplace = True)

        except:
            pass
        
        # Add scraped data to the database
        df.to_sql(table, self.db, if_exists = 'replace', index = False)
        
        self._close()
        
    def loadData(self, table):
        self._open()
        
        df = pd.read_sql_query(f"SELECT * FROM {table};", self.db)
        
        self._close()
        return df
    
    def dropTable(self, table):
        self._open()
        self.cursor.execute(f"DROP TABLE {table}")
        self._close()

In [91]:
# Configure the Riot API
api = riotApi(config.apiKey)

In [85]:
# Request summoner data and upload the data into the database
entryRequest = []

tiers = {'DIAMOND': 1}
ranks = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}

for tierKey, tierVal in tiers.items():
    for rankKey, rankVal in ranks.items():
        entryRequest += api.entryData(queueTypes[0], tierKey, rankKey)

# Get all leagues
leagues = pd.DataFrame(entryRequest)['leagueId']
leagues.drop_duplicates(inplace = True)

leagueRequest = []
for league in leagues:
    leagueRequest += api.leagueData(league)['entries']
    
summonerDF = pd.DataFrame(leagueRequest)
summonerDF = summonerDF.drop(columns = ['miniSeries'])
summonerDF.drop_duplicates(inplace = True)

db_lol = sqlDatabase('../data/test.db')
db_lol.appendData(summonerDF, 'SUMMONERS')

64
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [86]:
summonerNames = db_lol.loadData('SUMMONERS')['summonerName']
accountIds = []
for summonerName in summonerNames:
    accountIds += api.summonerData(summonerId)["accountId"]

gameIds = []
for accountId in accountIds:
    

summonerId    summonerName  \
0       04FKbAZuAjWuEGXFcvcfeAptmS5oSnSJORZPMgd3pfjOVfM         jeff506   
1       qlYZhauat6gtyJB8Xl09upjghBNLlwNFEGBZZCw7IS-exhA  King Kong Wong   
2       JrkcFnPrrj7J3mk6E0K2ShlspSS9LFnm4giMUE1ddSOq7jk     RYL Whlt3zZ   
3       wpKt3U8GSjvIl-w_Av-1J16QRCqLPXQCikCsrGEOTEqgtkQ            oSYo   
4      NXbKnED50pkl8Ec_lDRZf9VmRiLnvomeHuNmfcQ0ojU0A2XP         BUFF JG   
...                                                 ...             ...   
11760   Ml5GMQt8xueP9P39m_Rg6gg4qIZIf9vDC6r8U5gH_L5ZBzI        Quitzera   
11761  VVjXohITRc0MHdmZXIYfwmKEXWlAMEspvPcVVU_rpINgOQEP           Jrezx   
11762   04yOH-L5PnkS9e7VJAYcK9Bkgdsy7NoCktH_YstffwacQm8          ShepGG   
11763   kbSNeY8W8Xhe0eoypxOsUyPWjvWUBw6Bifxo-wPH5-zXFkM       AngryJack   
11764   8oEYwtSfup-3izOnkIDY42piQCPlJ1izQRe21KslPF2D90Q           Aegyo   

       leaguePoints rank  wins  losses  veteran  inactive  freshBlood  \
0                75   II    56      58        0         0           0   
1                59   IV    27      32        0         0           0   
2                75   IV    82      88        0         0           1   
3                69   IV    19      16        0         0           0   
4                61  III   173     159        1         0           0   
...             ...  ...   ...     ...      ...       ...         ...   
11760            47   IV    30      20        0         0           1   
11761             0   IV    38      44        0         0           1   
11762           100   IV    25      16        0         0           1   
11763            23   IV    25      20        0         0           1   
11764            11   IV    19      12        0         0           1   

       hotStreak  
0              0  
1              0  
2              1  
3              0  
4              0  
...          ...  
11760          0  
11761          0  
11762          1  
11763          0  
11764          0  

[11765 rows x 10 columns]

In [88]:
api.summonerData('Belgian Baron')

{'id': 'Ml434rS7FhjUFRs_7AH26xCUnBkXeqIPRKUhVZGcZYxCKN-8',
 'accountId': '5Qr7KUpgwRz1Ql9KEDqVKwEubhBGSQQ_Xf-9tK5PHfEAnegVgbWDggWA',
 'puuid': 'y1F3TDnLOPWWtA8Dn-xE9KSK-IzGRaHUWJ0XemPTPGFQR_fXYCV2rN_0QJ-xZpE8w_8qwpBemowGgw',
 'name': 'Belgian Baron',
 'profileIconId': 22,
 'revisionDate': 1580619720000,
 'summonerLevel': 48}

Get gameId from accountId and only look at the queue == 420 (ranked solo)

In [94]:
test = api.matchListData('5Qr7KUpgwRz1Ql9KEDqVKwEubhBGSQQ_Xf-9tK5PHfEAnegVgbWDggWA')['matches']

In [95]:
len(test)

100

In [96]:
test

[{'platformId': 'NA1',
  'gameId': 3283774096,
  'champion': 101,
  'queue': 450,
  'season': 13,
  'timestamp': 1580618510518,
  'role': 'DUO_SUPPORT',
  'lane': 'NONE'},
 {'platformId': 'NA1',
  'gameId': 3283719036,
  'champion': 16,
  'queue': 450,
  'season': 13,
  'timestamp': 1580617023878,
  'role': 'SOLO',
  'lane': 'MID'},
 {'platformId': 'NA1',
  'gameId': 3283713557,
  'champion': 76,
  'queue': 450,
  'season': 13,
  'timestamp': 1580615430281,
  'role': 'DUO_SUPPORT',
  'lane': 'MID'},
 {'platformId': 'NA1',
  'gameId': 3283645156,
  'champion': 17,
  'queue': 400,
  'season': 13,
  'timestamp': 1580612857774,
  'role': 'SOLO',
  'lane': 'TOP'},
 {'platformId': 'NA1',
  'gameId': 3283617131,
  'champion': 51,
  'queue': 400,
  'season': 13,
  'timestamp': 1580610715046,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'NA1',
  'gameId': 3283612902,
  'champion': 115,
  'queue': 420,
  'season': 13,
  'timestamp': 1580609309314,
  'role': 'DUO_SUPPORT',
  'lane'